# 2.3.3 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot, Chain-Of-Thought
* Best Ergebnisse (text_prompts, img_prompts): A2, [B2.3,C2.3,D3.3], E4.1

### Setup

In [1]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
samantha = OpenAI(instructions)
parser = Parser()
runner = Runner()

In [2]:
pdf_path = "../data/raw/bpläne/3_xplanung/BP_872A_AUFST_TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print("Seitenzahl:", len(pdf_prompts))

Seitenzahl: 66


### A) OCR komplettes PDF

In [3]:
instruction = 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
instruction_prompt = parser.text2prompts([instruction])

In [4]:
# A1) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Komplettes PDF auf einmal einlesen. Ziel: Konsistentes Layout.
pdf_text = samantha.request([*pdf_prompts, *instruction_prompt])
pprint(pdf_text)

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}
\usepackage{geometry}
\geometry{a4paper, margin=1in}

\title{Bebauungsplan Nr. 872 A}
\author{Stadt Augsburg}
\date{In Kraft getreten am: 23.02.2024}

\begin{document}

\maketitle

\section*{Aufstellung}
\textbf{Bebauungsplan Nr. 872 A} \\
„Zwischen Waldstraße und Döllgaststraße“ \\
mit integriertem Grünordnungsplan


ewpage

\tableofcontents


ewpage

\section*{Abkürzungen}
\begin{tabular}{ll}
Abkürzung & Erläuterung \\
BauGB & Baugesetzbuch \\
BauNVO & Baunutzungsverordnung \\
BayBO & Bayerische Bauordnung \\
BayNatSchG & Bayerisches Naturschutzgesetz \\
BBodSchV & Bundesbodenschutzverordnung \\
BHKW & Blockheizkraftwerk \\
BImSchG & Bundes-Immissionsschutzgesetz \\
BNatSchG & Bundesnaturschutzgesetz \\
BP & Bebauungsplan \\
dB(A) & Dezibel (A-bewertet) – logarithmische Maßeinheit, die den Schalldruckpegel angibt \\
DIN 18005 & Deutsche Industrienorm zum Schallschutz im Städtebau \\
DIN 18920 & Deutsch

In [5]:
# A2) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten. Ziel: Fokus maximieren und Context-Window klein halten + Seitenzahlen bleiben erhalten.
prompt_chain = list(map(lambda prompt: samantha.lambdaRequest([prompt, *instruction_prompt]), pdf_prompts))
msgbp872apdf = await asyncio.gather(*prompt_chain)
pprint(msgbp872apdf)
%store msgbp872apdf

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\begin{center}
    \includegraphics[width=0.2\textwidth]{logo.png} \\
    \vspace{1cm}
    \textbf{\huge Stadt Augsburg} \\
    \vspace{2cm}
    \textbf{\LARGE Aufstellung} \\
    \vspace{1cm}
    \textbf{\Huge Bebauungsplan Nr. 872 A} \\
    \vspace{0.5cm}
    \textbf{\LARGE „Zwischen Waldstraße und Döllgaststraße“} \\
    \vspace{0.5cm}
    \textbf{\large mit integriertem Grünordnungsplan} \\
    \vspace{3cm}
    \textbf{\large Textteil} \\
    \vspace{2cm}
    \textbf{\large In Kraft getreten am:} \\
    \vspace{0.5cm}
    \textbf{\huge 23.02.2024} \\
\end{center}

\end{document}
```
#############################################
```latex
\documentclass{article}
\usepackage[utf8]{inputenc}

\begin{document}

\section*{Inhaltsverzeichnis}

\begin{enumerate}
    \item Inhaltsverzeichnis \dotfill 2
    \item Abkürzungen \dotfill 3
    \item Ermächtigungsgrundlage \dotfill 5
    \item A.

### B) Art der baulichen Nutzung

1. text_prompts
2. img_prompts

In [3]:
%store -r msgbaunvo_art msgbp872apdf
context_art = parser.text2prompts([msgbaunvo_art])

In [4]:
# B1) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zum Thema "Art der baulichen Nutzung". '

def run():
    return samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_art)
results = runner.consistency_check(run)
msg233_bp872_tt_art = results[0][0]
%store msg233_bp872_tt_art

['Hier sind die extrahierten textlichen Festsetzungen und Begründungen zum Thema "Art der baulichen Nutzung" aus dem bereitgestellten LaTeX-Dokument:

```latex
\\documentclass{article}
\\usepackage[utf8]{inputenc}
\\usepackage{graphicx}

\\begin{document}

\\section*{D.5.2. Begründung einzelner Festsetzungen}

\\subsection*{D.5.2.1. Art der baulichen Nutzung}

\\subsubsection*{D.5.2.1.1. Allgemeines Wohngebiet}

Nachdem das aktuelle Planungskonzept vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebiets vorsieht, wird anstatt der im rechtskräftigen BP Nr. 872 festgesetzten Fläche für den Gemeinbedarf mit der Zweckbestimmung „Gesundheit/Fürsorge“ künftig ein allgemeines Wohngebiet (WA) gemäß § 4 BauNVO für das Plangebiet festgesetzt. Die Festsetzungen zu den im Planbereich ausgewiesenen Wohnnutzungen orientieren sich dabei am Nutzungskatalog der allgemeinen zulässigen Wohnnutzungen des § 4 Abs. 2 BauNVO. Somit kann die insbesondere im südwestlich und teilweise auch östlich benach

In [5]:
# B2.1) Textinformationen: Relevante Seiten extrahieren (Vorverarbeitung) – WITHOUT CONTEXT
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die textliche Festsetzungen und Begründungen zum Thema "Art der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}.'
async def run():
    return await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
await runner.async_consistency_check(run)

{"Seiten": [6, 26, 27, 28]}
#############################################
```json
{"Seiten": [6, 26, 27, 28]}
```
#############################################
```json
{"Seiten": [6, 26, 27, 28]}
```
#############################################


['{"Seiten": [6, 26, 27, 28]}',
 '```json\n{"Seiten": [6, 26, 27, 28]}\n```',
 '```json\n{"Seiten": [6, 26, 27, 28]}\n```']

In [6]:
# B2.2) Textinformationen: Relevante Seiten extrahieren (Vorverarbeitung) – WITH CONTEXT
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die textliche Festsetzungen und Begründungen zum Thema "Art der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}.'
async def run():
    return await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_art], img_type="pdf")
await runner.async_consistency_check(run)

['```json
{"Seiten": [6, 7, 8, 26, 27, 28]}
```']
#############################################
['{"Seiten": [6, 7, 8, 26, 27, 28]}']
#############################################
['```json
{"Seiten": [6, 7, 8, 26, 27, 28, 29]}
```']
#############################################


[['```json\n{"Seiten": [6, 7, 8, 26, 27, 28]}\n```'],
 ['{"Seiten": [6, 7, 8, 26, 27, 28]}'],
 ['```json\n{"Seiten": [6, 7, 8, 26, 27, 28, 29]}\n```']]

In [7]:
# B2.3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten
pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zum Thema "Art der baulichen Nutzung". '

def run():
    return samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_art)
results = runner.consistency_check(run)

### C.2. Planungsrechtliche Festsetzungen durch Text

#### § 4 Art der baulichen Nutzung

(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

**Zulässig sind:**
- Wohngebäude,
- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

**Nicht zulässig sind:**
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### D.5.2. Begründung einzelner Festsetzungen

#### D.5.2.1. Art der baulichen Nutzung

##### D.5.2.1.1. Allgemeines Wohngebiet

Nachdem das aktuelle Planungskonzept vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebietes vorsieht, wird anstatt der im rechtskräftigen BP Nr. 

In [8]:
# B3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF

instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zum Thema "Art der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>). '

async def run():
    return await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_art], img_type="pdf")
await runner.async_consistency_check(run)

['### Art der baulichen Nutzung: 
#### Textliche Festsetzungen:
1. **§ 4 Art der baulichen Nutzung**
   - Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
   - Zulässig sind:
     - Wohngebäude,
     - die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
     - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.
   - Nicht zulässig sind:
     - Betriebe des Beherbergungsgewerbes,
     - sonstige nicht störende Gewerbebetriebe,
     - Anlagen für Verwaltungen,
     - Gartenbaubetriebe,
     - Tankstellen.
   - Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

#### Begründungen:
1. **D.5.2.1. Art der baulichen Nutzung**
   - **D.5.2.1.1. Allgemeines Wohngebiet**
     - Das aktuelle Planungskonzept sieht vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebietes vor, anstatt der im r

[['### Art der baulichen Nutzung: \n#### Textliche Festsetzungen:\n1. **§ 4 Art der baulichen Nutzung**\n   - Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.\n   - Zulässig sind:\n     - Wohngebäude,\n     - die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,\n     - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.\n   - Nicht zulässig sind:\n     - Betriebe des Beherbergungsgewerbes,\n     - sonstige nicht störende Gewerbebetriebe,\n     - Anlagen für Verwaltungen,\n     - Gartenbaubetriebe,\n     - Tankstellen.\n   - Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.\n\n#### Begründungen:\n1. **D.5.2.1. Art der baulichen Nutzung**\n   - **D.5.2.1.1. Allgemeines Wohngebiet**\n     - Das aktuelle Planungskonzept sieht vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebietes v

### C) Maß der baulichen Nutzung

1. text_prompts
2. img_prompts

In [9]:
%store -r msgbaunvo_maß msgbp872apdf
context_maß = parser.text2prompts([msgbaunvo_maß])

In [10]:
# C1) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zum Thema "Maß der baulichen Nutzung". '
def run():
    return samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_maß)
results = runner.consistency_check(run)
msg233_bp872_tt_maß = results[0][0]
%store msg233_bp872_tt_maß

['Hier sind die textlichen Festsetzungen und Begründungen zum Thema "Maß der baulichen Nutzung" aus den bereitgestellten LaTeX-Dokumenten extrahiert:

### Textliche Festsetzungen

#### § 5 Maß der baulichen Nutzung

(1) Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.

(2) In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
- WA1: GRZ maximal 0,65,
- WA2: GRZ maximal 0,80,
- WA3: GRZ maximal 0,75,
- WA4: GRZ maximal 0,75.

#### § 6 Höhenlage, Bezugspunkte

(2) Die zulässigen Gebäudeoberkanten (OK) sind für die einzelnen Bereiche in der Planzeichnung (Teil A) festgesetzt. Gebäudeoberkante ist der jeweils oberste Abschluss der Außenwand (Attika). Bezugspunkt ist die jeweils festgesetzte Höhenkote des Erdgeschossfertigfußbodens. Bei der zwingend festgesetzten Gebäudeoberkante in den Baufeldern 4

In [11]:
# C2.1) Textinformationen: Relevante Seiten extrahieren (Vorverarbeitung) – WITHOUT CONTEXT
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die textliche Festsetzungen und Begründungen zum Thema "Maß der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}.'
async def run():
    return await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
await runner.async_consistency_check(run)

```json
{"Seiten": [6, 7, 8, 26, 27, 28, 29]}
```
#############################################
```json
{"Seiten": [6, 7, 28, 29]}
```
#############################################
```json
{"Seiten": [6, 28, 29]}
```
#############################################


['```json\n{"Seiten": [6, 7, 8, 26, 27, 28, 29]}\n```',
 '```json\n{"Seiten": [6, 7, 28, 29]}\n```',
 '```json\n{"Seiten": [6, 28, 29]}\n```']

In [12]:
# C2.2) Textinformationen: Relevante Seiten extrahieren (Vorverarbeitung) – WITH CONTEXT
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die textliche Festsetzungen und Begründungen zum Thema "Maß der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}.'
async def run():
    return await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_maß], img_type="pdf")
await runner.async_consistency_check(run)

['```json
{"Seiten": [6, 7, 8, 26, 27, 28, 29, 30, 31, 32, 33]}
```']
#############################################
['```json
{"Seiten": [6, 7, 8, 28, 29, 30]}
```']
#############################################
['```json
{"Seiten": [6, 7, 8, 28, 29, 30]}
```']
#############################################


[['```json\n{"Seiten": [6, 7, 8, 26, 27, 28, 29, 30, 31, 32, 33]}\n```'],
 ['```json\n{"Seiten": [6, 7, 8, 28, 29, 30]}\n```'],
 ['```json\n{"Seiten": [6, 7, 8, 28, 29, 30]}\n```']]

In [13]:
# C2.3) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten
pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zum Thema "Maß der baulichen Nutzung". '

def run():
    return samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_maß)
results = runner.consistency_check(run)
msg233_maß = results[0]
%store msg233_maß

Hier sind die textlichen Festsetzungen und Begründungen zum Thema "Maß der baulichen Nutzung" aus den bereitgestellten Seiten des Bebauungsplans:

### C.2. Planungsrechtliche Festsetzungen durch Text

#### § 5 Maß der baulichen Nutzung

(1) Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.

(2) In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
- WA1: GRZ maximal 0,65,
- WA2: GRZ maximal 0,80,
- WA3: GRZ maximal 0,75,
- WA4: GRZ maximal 0,75.

### D.5.2. Begründung einzelner Festsetzungen

#### D.5.2.1. Art der baulichen Nutzung

##### D.5.2.1.2. Fremdwerbeanlagen

Die als gewerbliche Anlagen einzuordnenden Fremdwerbeanlagen werden im gesamten Geltungsbereich des BP Nr. 872 A ausgeschlossen, da sie zu einer Beeinträchtigung des Stadtbildes und der auch im Umfeld bestehenden Wohn- und Kliniknu

### D) Bauweise, überbaubare Grundstücksflächen

1. text_prompts
2. img_prompts

In [14]:
%store -r msgbaunvo_bg msgbp872apdf
context_bg = parser.text2prompts([msgbaunvo_bg])

In [15]:
# D1) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zum Thema "Bauweise, überbaubare Grundstücksflächen". '

def run():
    return samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_bg)
results = runner.consistency_check(run)
msg233_bp872_tt_bp = results[0][0]
%store msg233_bp872_tt_bp

['### Textliche Festsetzungen zum Thema "Bauweise, überbaubare Grundstücksflächen"

#### § 7 Bauweise, Überbaubare Grundstücksflächen, Abstandsflächen

(1) Sofern erforderlich, ist die Bauweise in der Planzeichnung (Teil A) festgesetzt.

(2) Die überbaubaren Grundstücksflächen sind durch Baulinien und Baugrenzen in der Planzeichnung (Teil A) festgesetzt.

(3) Vorbauten dürfen die Baugrenzen bis zu 1,5 m auf einer Breite von maximal 5,0 m je Eingang überschreiten. Balkone dürfen mit Ausnahme des jeweils obersten Geschosses die Baugrenzen bis zu 0,90 m auf einer Breite von maximal 7,50 m Balkon überschreiten. Zwischen den einzelnen Balkonen muss ein Abstand von mindestens 3,25 m eingehalten werden.

(4) Terrassen an Gebäuden dürfen sich bis zu einer Tiefe von 2,5 m auch auf Flächen außerhalb der überbaubaren Grundstücksflächen erstrecken. Sie dürfen außerhalb der überbaubaren Grundstücksflächen nicht zulässig sein, da in diesen Bereichen zum Fabrikkanal hin eine möglichst ruhige Fassaden

In [16]:
# D2.1) Textinformationen: Relevante Seiten extrahieren (Vorverarbeitung) – WITHOUT CONTEXT
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die textliche Festsetzungen und Begründungen zum Thema "Bauweise, überbaubare Grundstücksflächen" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}.'
async def run():
    return await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
await runner.async_consistency_check(run)

```json
{"Seiten": [7, 30, 31]}
```
#############################################
```json
{"Seiten": [6, 7, 8, 30, 31, 32]}
```
#############################################
```json
{
  "Seiten": [7, 30, 31, 32]
}
```
#############################################


['```json\n{"Seiten": [7, 30, 31]}\n```',
 '```json\n{"Seiten": [6, 7, 8, 30, 31, 32]}\n```',
 '```json\n{\n  "Seiten": [7, 30, 31, 32]\n}\n```']

In [17]:
# D2.2) Textinformationen: Relevante Seiten extrahieren (Vorverarbeitung) – WITH CONTEXT
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die textliche Festsetzungen und Begründungen zum Thema "Bauweise, überbaubare Grundstücksflächen" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}.'
async def run():
    return await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_bg], img_type="pdf")
await runner.async_consistency_check(run)

['```json
{"Seiten": [6, 7, 8, 9, 10]}
```']
#############################################
['```json
{"Seiten": [6, 7, 8]}
```']
#############################################
['```json
{"Seiten": [6, 7, 8]}
```']
#############################################


[['```json\n{"Seiten": [6, 7, 8, 9, 10]}\n```'],
 ['```json\n{"Seiten": [6, 7, 8]}\n```'],
 ['```json\n{"Seiten": [6, 7, 8]}\n```']]

In [18]:
# D2.3)Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten
pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zum Thema "Bauweise, überbaubare Grundstücksflächen". '

def run():
    return samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_bg)
results = runner.consistency_check(run)
msg233_bg = results[0]
%store msg233_bg

### Textliche Festsetzungen zum Thema "Bauweise, überbaubare Grundstücksflächen"

#### § 7 Bauweise, Überbaubare Grundstücksflächen, Abstandsflächen
(1) Sofern erforderlich, ist die Bauweise in der Planzeichnung (Teil A) festgesetzt.

(2) Die überbaubaren Grundstücksflächen sind durch Baulinien und Baugrenzen in der Planzeichnung (Teil A) festgesetzt.

(3) Vorbauten dürfen die Baugrenzen bis zu 1,5 m auf einer Breite von maximal 5,0 m je Eingang überschreiten. Balkone dürfen mit Ausnahme des jeweils obersten Geschosses die Baugrenzen bis zu 0,90 m auf einer Breite von maximal 7,50 m je Balkon überschreiten. Zwischen den einzelnen Balkonen muss ein Abstand von mindestens 3,25 m eingehalten werden.

(4) Terrassen an Gebäuden dürfen sich bis zu einer Tiefe von 2,5 m auch auf Flächen außerhalb der überbaubaren Grundstücksflächen erstrecken. Sie dürfen außerhalb der überbaubaren Grundstücksflächen nicht durch Wintergärten oder ähnliches überbaut werden.

(5) Im Kronenbereich der in der Plan

### E) Alle Themen in einem Prompt

1. text_prompts (TODO)
2. img_prompts

In [19]:
%store -r msgbaunvo msgbp872apdf
context_all = parser.text2prompts([msgbaunvo])

In [20]:
# E1) Textinformationen: Alle Themen – WITH CONTEXT – IMG_PROMPTS
instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zu den folgenden Themen: "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise, überbaubare Grundstücksflächen". '

messages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_all], img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung

#### Textliche Festsetzungen
**§ 4 Art der baulichen Nutzung**
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

Zulässig sind:
- Wohngebäude,
- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

Nicht zulässig sind:
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

#### Begründung
**D.5.2.1. Art der baulichen Nutzung**
**D.5.2.1.1. Allgemeines Wohngebiet**
Nach dem aktuellen Planungskonzept wird eine wohnbauliche Nutzung innerhalb des Plangebiets vorgesehen. Die Festsetzungen zu den im Planbereich ausgewiesenen Wohnnutzungen orientieren sich dabei

In [21]:
# E2.1) Textinformationen: Alle Themen – WITHOUT CONTEXT – IMG_PROMPTS
# Problem: Kontext ist sehr groß, sodass GPT den Überblick verliert und Informationen verliert. 
instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zu den folgenden Themen: "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise, überbaubare Grundstücksflächen". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>). '

messages = await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung:
- Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
  - Zulässig sind:
    - Wohngebäude,
    - die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe,
    - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.
  - Nicht zulässig sind:
    - Betriebe des Beherbergungsgewerbes,
    - sonstige nicht störende Gewerbebetriebe,
    - Anlagen für Verwaltungen,
    - Gartenbaubetriebe,
    - Tankstellen.
- Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung:
- Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
- In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ übersch

In [22]:
# E2.2) Mehrere Threads – WITHOUT CONTEXT
# Idee: Jede Seite einzeln verarbeiten, um Context-Window klein zu halten.
# Problem: Informationen sind auf mehrere Seiten verteilt => fehlender Kontext => unvollständige Resultate.
instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zu den folgenden Themen: "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise, überbaubare Grundstücksflächen". Steht keine relevante Information zur Verfügung gebe "-" aus. '

async def run(pdf_prompt):
    return samantha.request([{"type": "text", "text": instruction},pdf_prompt])
prompt_chain = list(map(lambda pdf_prompt: run(pdf_prompt), pdf_prompts))
results = await asyncio.gather(*prompt_chain)
pprint(results)

Um die gewünschten Informationen zu extrahieren, benötige ich den vollständigen Textteil des Bebauungsplans. Bitte lade die entsprechenden Seiten hoch, die die textlichen Festsetzungen und Begründungen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung" und "Bauweise, überbaubare Grundstücksflächen" enthalten.
#############################################
Hier sind die textlichen Festsetzungen und Begründungen zu den angegebenen Themen aus dem Inhaltsverzeichnis des Bebauungsplans Nr. 872 A, Aufstellung:

**Art der baulichen Nutzung:**
- C.2. Planungsrechtliche Festsetzungen durch Text (Seite 6)
- D.2.3. Städtebauliche Strukturen und Nutzungen im Plangebiet (Seite 18)
- D.2.4. Städtebauliche Strukturen und Nutzungen in der Umgebung (Seite 19)

**Maß der baulichen Nutzung:**
- D.2.1. Geltungsbereich, Lage, Größe (Seite 17)
- D.2.2. Eigentumsverhältnisse (Seite 18)
- D.5.2. Ziele und Zwecke der Planung, Planungskonzept (Maß der baulichen Nutzung) (Seite 26)

**Bauweise

In [23]:
# E3) Textinformationen: alle Themen – WITH CONTEXT – TEXT_PROMPTS
# Vector Store
instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zu den folgenden Themen: "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise, überbaubare Grundstücksflächen". '

def run():
    return samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_bg)
results = runner.consistency_check(run)

['### Textliche Festsetzungen

#### § 4 Art der baulichen Nutzung
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

**Zulässig sind:**
- Wohngebäude,
- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

**Nicht zulässig sind:**
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

#### § 5 Maß der baulichen Nutzung
(1) Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.

(2) In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
- WA1: GRZ maximal 0,65,
- WA2: 

In [32]:
# E4.1) Ein Thread – Relevante Seiten extrahieren (Vorverarbeitung) – (Bestes Ergebnis) – WITHOUT CONTEXT
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zu folgenden Themen beinhalten: "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise und überbaubare Grundstücksflächen". JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}.'

async def run():
    return await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
await runner.async_consistency_check(run)
pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all

# Wenn Dokument zu groß wird: z.B. chunk_size=10 pages, overlap=2 pages.
# chunk_size = 10, overlap = 2
# n = math.ceil(len(pdf_prompts)/chunk_size)
# chunks = list(map(lambda i: pdf_prompts[slice(max(i*chunk_size-overlap,0),i*chunk_size+chunk_size)], range(n)))

```json
{"Seiten": [6, 7, 8]}
```
#############################################
```json
{
  "Seiten": [6, 7, 8, 9, 30, 31, 32, 33]
}
```
#############################################
```json
{
  "Seiten": [6, 7, 8, 29, 30, 31, 32, 33]
}
```
#############################################


In [31]:
# E4.2) Ein Thread – Relevante Seiten extrahieren (Vorverarbeitung) – (Bestes Ergebnis) – WITH CONTEXT
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zu folgenden Themen beinhalten: "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise, überbaubare Grundstücksflächen". JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}.'

async def run():
    return await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_all], img_type="pdf")
await runner.async_consistency_check(run)
pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all

['```json
{
  "Seiten": [6, 7, 8]
}
```']
#############################################
['```json
{
  "Seiten": [6, 7, 8]
}
```']
#############################################
['```json
{
  "Seiten": [6, 7, 8]
}
```']
#############################################


In [26]:
# E4.3) Textinformationen: alle Themen – WITH CONTEXT – IMG_PROMPTS (Vorgefilterte Seiten)
instruction = 'Extrahiere alle textlichen Festsetzungen und Begründungen zu den folgenden Themen: "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise und überbaubare Grundstücksflächen". '

def run():
    return  samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_all)
results = runner.consistency_check(run)
msg233_all = results[0]
%store msg233_all


### Textliche Festsetzungen

#### Art der baulichen Nutzung (§ 4)
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

**Zulässig sind:**
- Wohngebäude,
- der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

**Nicht zulässig sind:**
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

#### Maß der baulichen Nutzung (§ 5)
(1) Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.

(2) In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu fol